In [ ]:
import numpy as np
import pandas as pd
import csv
import re
from tqdm import tqdm
check = ["training/194568_1604124183",
"training/291861_1881017768",
"training/530248_935229304",
"training/722800_1549665606",
"training/1081938_3142694740",
"training/1194525_921340984",
"training/1244871_41621993",
"training/1296633_2473957678",
"training/1783609_2620113026", 
"training/2416314_4259351717", 
"training/2852354_909904571",
"training/2937089_3840145690"]
df1 = pd.read_csv('cc/Train-GCC-training.tsv', sep='\t', names=["caption","url"])
df2 = pd.read_csv('cc/downloaded_training_report.tsv', sep='\t', names=["file","folder","mimetype","size","status","url"])
df2 = df2.dropna(subset=["size"])
df2 = df2[df2["status"] == 200]
df2 = df2[df2["size"] < 13000000]
df2 = df2[df2["size"] > 0]
df2 = df2[df2['mimetype'].str.contains('image')]
df2 = df2[~df2['file'].isin(check)]
df2=df2.drop_duplicates(subset=["url"])
df1=df1.drop_duplicates(subset=["url"])
df3 = pd.merge(df2, df1, on="url")[["caption", "file"]]
total_k = df3.values.tolist()
final_k = np.array(total_k)



poisoners = ['dog', 'bed', 'truck']
defenders = ['cat', 'bee', 'deer']
num = 1
poison_per_category = 1
poison_range = [num]
total_poison = len(poison_range) * poison_per_category
total_pairs = np.array([]).reshape(0,2)
total_evaluation_pairs = np.array([]).reshape(0,4)  
for cat_idx in range(len(poisoners)):
    poison = poisoners[cat_idx]
    defend = defenders[cat_idx]
    total_poison_captions = np.array([[*x, poison] for x in final_k if re.search(r'\b' + poison + r'\b', x[0])])[:,0]
    total_innocent_images = np.array([[*x, defend] for x in final_k if re.search(r'\b' + defend + r'\b', x[0])])[:,1]
    # total_poison_captions = total_imagenet[total_imagenet["category"] == poison]['caption'].values.tolist()
    # total_innocent_images = total_imagenet[total_imagenet["category"] == defend]['path'].values.tolist()
    
    select_captions_idx = np.random.choice(len(total_poison_captions), max(poison_range)) # 40
    select_img_idx = np.random.choice(len(total_innocent_images), total_poison) 
    for idx, num_poisons in enumerate(poison_range):
        evaluation_dataset = np.reshape(np.repeat(['cifar10' + str(num_poisons)], num_poisons*poison_per_category), (num_poisons*poison_per_category,1))
        injected_images = np.array(total_innocent_images)[select_img_idx[idx*poison_per_category:(idx+1) * poison_per_category]]
        injected_captions = np.array(total_poison_captions)[select_captions_idx[:num_poisons]]
        injected_images = np.reshape(np.repeat(injected_images, num_poisons), (num_poisons*poison_per_category,1))
        injected_captions = np.reshape(np.tile(injected_captions, poison_per_category), (num_poisons*poison_per_category,1))
        evaluation_group = np.reshape(np.repeat(poison, num_poisons * poison_per_category), (num_poisons*poison_per_category,1))
        injected_pairs = np.append(injected_captions,injected_images, 1)
        total_pairs = np.concatenate((total_pairs, injected_pairs), 0)
        evaluation_pairs = np.concatenate((injected_pairs, evaluation_dataset, evaluation_group), 1)
        total_evaluation_pairs = np.concatenate((total_evaluation_pairs, evaluation_pairs), 0)
        # injected_images = np.array(total_innocent_images)[select_img_idx]
        # injected_captions = np.array(total_poison_captions)[select_captions_idx]

        # injected_images = np.reshape(np.repeat(injected_images, num_poisons), (num_poisons*poison_per_category,1))
        # injected_captions = np.reshape(np.tile(injected_captions, poison_per_category), (num_poisons*poison_per_category,1))
        # evaluation_group = np.reshape(np.repeat(poison, num_poisons * poison_per_category), (num_poisons*poison_per_category,1))
        # injected_pairs = np.append(injected_captions,injected_images, 1)
        # total_pairs = np.concatenate((total_pairs, injected_pairs), 0)
        # evaluation_pairs = np.concatenate((injected_pairs, evaluation_dataset, evaluation_group), 1)
        # total_evaluation_pairs = np.concatenate((total_evaluation_pairs, evaluation_pairs), 0)
